### Tool Calling in LangChain

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
from dotenv import load_dotenv

In [2]:
# tool creation
@tool
def multiply(x: int, y: int) -> int:
    """Given two numbers, multiply them and returns the product."""
    return x * y

In [3]:
print(multiply.invoke({"x": 2, "y": 3}))

6


In [4]:
multiply.name

'multiply'

In [5]:
multiply.description

'Given two numbers, multiply them and returns the product.'

In [6]:
multiply.args

{'x': {'title': 'X', 'type': 'integer'},
 'y': {'title': 'Y', 'type': 'integer'}}

#### Tool Binding

In [7]:
load_dotenv()
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [8]:
llm_with_tools = llm.bind_tools([multiply])

In [9]:
llm_with_tools.invoke("Hi how are you?")

AIMessage(content="Hello! I'm here and ready to assist you. How can I help you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 57, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CWVW6d38xvxoBrXv0tSaLI4ocCELy', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--4a1c76b7-f242-4a93-bb6b-8bc5b63985e1-0', usage_metadata={'input_tokens': 57, 'output_tokens': 19, 'total_tokens': 76, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
output = llm_with_tools.invoke("can you multiply 2 and 10?")

print(output)

content='' additional_kwargs={'tool_calls': [{'id': 'call_yjihF9FQd3t5Y94wmOvb8aXz', 'function': {'arguments': '{"x":2,"y":10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 61, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CWVWudK97njhapR2uh4Tn93F26gZB', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--5cb9ee14-3504-4b53-8500-22ebe32baa2b-0' tool_calls=[{'name': 'multiply', 'args': {'x': 2, 'y': 10}, 'id': 'call_yjihF9FQd3t5Y94wmOvb8aXz', 'type': 'tool_call'}] usage_metadata={'input_tokens': 61, 'output_tokens': 17, 'total_tokens': 78, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_d

In [18]:
arguments = output.tool_calls[0]['args']

In [ ]:
multiply.invoke(arguments) # this is the same as multiply.invoke({"x": 2, "y": 10}) 

# this gives just the result (in our case it is the product of 2 and 10)

20

In [ ]:
# let us see what happens when we call the tool from the tool_call
multiply.invoke(output.tool_calls[0])

# this is nicely formatted in a ToolMessage

ToolMessage(content='20', name='multiply', tool_call_id='call_yjihF9FQd3t5Y94wmOvb8aXz')

In [ ]:
# let us organize the above steps better
messages = []

# setup the prompt for the LLM
query = HumanMessage(content="can you multiply 2 and 10?")

# append the prompt
messages.append(query)

# invoke the LLM with the message
output = llm_with_tools.invoke(messages)

# append the output
messages.append(output)

# invoke the tool using the tool_call
tool_result = multiply.invoke(output.tool_calls[0])

# append the result
messages.append(tool_result)

print(messages)



[HumanMessage(content='can you multiply 2 and 10?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_gdm6XU6PCuFWISNgWOWKFqqp', 'function': {'arguments': '{"x":2,"y":10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 61, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CWVqzVxBlOKZgVTBZuOltIW53yrET', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ac815848-99ae-4033-a085-388cd2097147-0', tool_calls=[{'name': 'multiply', 'args': {'x': 2, 'y': 10}, 'id': 'call_gdm6XU6PCuFWISNgWOWKFqqp', 'type': 'tool_call'}], usage_metadata={'input_tokens': 61

In [ ]:
# invoke the LLM with all the messages at once 
llm_with_tools.invoke(messages).content

'The product of 2 and 10 is 20.'